In [1]:
import pandas as pd
import numpy as np

In [2]:
from datasets import Dataset, Image as dsImage, load_from_disk, ClassLabel

In [3]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True #imagenes ligeramente corruptas OK

In [4]:
df=pd.read_feather("Reclassified.feather")

In [5]:
df.head(5)

,Path,y,ny,my
0,images/2000/abr/p0423001.jpg,INA,INA,INA
1,images/2000/abr/p0423002.jpg,FUM,FUM,FUM
2,images/2000/abr/p0423003.jpg,FUM,FUM,FUM
3,images/2000/abr/p0424001.jpg,FUM,FUM,FUM
4,images/2000/abr/p0424002.jpg,FUM,FUM,FUM


In [6]:
def generate():
    for i, row in df.iterrows():
        try:
            Image.open(row["Path"])
            yield {
                "image":row["Path"],
                "labels":row["ny"],
            }
        except:
            print(row["Path"])
            continue

dataset=Dataset.from_generator(generate).cast_column("image",dsImage()).cast_column("labels",ClassLabel(names=df["ny"].unique().tolist())).train_test_split(test_size=.15, seed=36918, stratify_by_column="labels")
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'labels'],
        num_rows: 18632
    })
    test: Dataset({
        features: ['image', 'labels'],
        num_rows: 3289
    })
})

In [7]:
np.unique(dataset["train"]["labels"], return_counts=True)

(array([0, 1, 2, 3, 4]), array([2562, 7628, 1312,  238, 6892]))

In [8]:
np.unique(dataset["test"]["labels"], return_counts=True)

(array([0, 1, 2, 3, 4]), array([ 452, 1346,  232,   42, 1217]))

In [9]:
dataset.save_to_disk("popocatepetl-dataset/",num_proc=12)

Saving the dataset (0/12 shards):   0%|          | 0/18632 [00:00<?, ? examples/s]

Saving the dataset (0/12 shards):   0%|          | 0/3289 [00:00<?, ? examples/s]

In [12]:
dataset["train"].features["labels"].names

['INA', 'FUM', 'UNK', 'EXP', 'EXP+FUM']